<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 1



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [50]:
using LinearAlgebra, Printf, Plots

Esse trecho de código é usado apenas para garantir a repetibilidade dos números aleatórios gerados.

In [24]:
using Random
Random.seed!(42)

TaskLocalRNG()

## Questão 1

**Gradiente com backtracking**

In [25]:
function steepest_descent_backtracking(fun, x0; sigma=1e-4, rho=0.5, max_iter=2000, tol=1e-6)
  x = copy(x0)

  for k = 0:max_iter-1
    f, g, _ = fun(x)
    gnorm = norm(g)

    if gnorm <= tol
      return x, k, gnorm
    end

    # backtracking
    t = 1.0
    xn = x - t*g
    fn, _, _= fun(xn)

    while fn > f - sigma*t*gnorm^2
      t *= rho
      xn = x - t*g
      fn, _, _ = fun(xn)
    end

    x = xn
  end

  _, g, _ = fun(x)
  gnorm = norm(g)
  return x, max_iter, gnorm
end


steepest_descent_backtracking (generic function with 1 method)

**Newton com backtracking**

O método de Newton é dado por
$$
  x_{k+1} = x_k - \alpha_k H(x_k)^{-1} \nabla f(x_k)
$$
em que
- $\nabla f(x_k)$: gradiente de f;
- $H(x_k)$: matriz hessiana de f;
- $\alpha_k$: tamanho do passo.

In [26]:
function newton_backtracking(fun, x0; sigma=1e-4, rho=0.5, max_iter=2000, tol=1e-6, fallback=nothing)
  x = copy(x0)

  for k = 0:max_iter-1
    f, g, H = fun(x)
    gnorm = norm(g)

    if gnorm <= tol
      return x, k, gnorm
    end

    p = -(H \ g)

    # se não for direção de descida, usa fallback se existir
    if fallback != nothing && g' * p >= 0
      p = fallback(g, H)
    end

    # backtracking
    t = 1.0
    xn = x + t*p
    fn, _, _ = fun(xn)

    while fn > f + sigma*t*(g' * p)
        t *= rho
        xn = x + t*p
        fn, _, _ = fun(xn)
    end

    x = xn
  end

  f, g, H = fun(x)
  return x, max_iter, norm(g)
end

newton_backtracking (generic function with 1 method)

## Questão 2

Considere a função de Rosenbrock

In [27]:
function rosenb(x)
  a = 1.0
  b = 10.0

  f = (a - x[1])^2 + b*(x[2] - x[1]^2)^2

  grad_f = [ -2*(a - x[1]) + 2*b*(x[2] - x[1]^2)*(-2*x[1]),
        2*b*(x[2] - x[1]^2)]

  hess_f = [
        2 - 4*b*(x[2] - x[1]^2) + 8*b*x[1]^2   -4*b*x[1];
        -4*b*x[1]                               2*b
  ]

  return f, grad_f, hess_f
end

rosenb (generic function with 1 method)

Comparemos ambos os métodos anteriores considerando os pontos iniciais $x_0 = (1.01, 1.01)$ e $x_1 = (-1.0, 1.2)$

In [28]:
x0 = [1.01, 1.01]

x, max_iter_sd, gnorm_sd = steepest_descent_backtracking(rosenb, x0)
@printf("Gradiente:\n  Número de iterações: %d\n  Gradiente final: %.6e\n  Ponto final: [%.6f, %.6f]\n\n",
        max_iter_sd, gnorm_sd, x[1], x[2])

x, max_iter_n, gnorm_n = newton_backtracking(rosenb, x0)
@printf("Newton:\n  Número de iterações: %d\n  Gradiente final (norma): %.6e\n  Ponto final: [%.6f, %.6f]\n\n",
        max_iter_n, gnorm_n, x[1], x[2])


Gradiente:
  Número de iterações: 1027
  Gradiente final: 9.818612e-07
  Ponto final: [1.000001, 1.000002]

Newton:
  Número de iterações: 3
  Gradiente final (norma): 4.895830e-10
  Ponto final: [1.000000, 1.000000]



Para $x_0 = (1.01, 1.01)$ próximo do mínimo $x^* = (1,1)$, o método de Newton converge quadraticamente, resultando em apenas 3 iterações, enquanto o método do gradiente precisou de 1027 iterações. Além disso, o método de Newton foi bem mais preciso quando comparado ao método do gradiente.

In [29]:
x1 = [-1.0, 1.2]

x, max_iter_sd, gnorm_sd = steepest_descent_backtracking(rosenb, x1)
@printf("Gradiente:\n  Número de iterações: %d\n  Gradiente final: %.6e\n  Ponto final: [%.6f, %.6f]\n\n",
        max_iter_sd, gnorm_sd, x[1], x[2])

x, max_iter_n, gnorm_n = newton_backtracking(rosenb, x1, max_iter=5000)
@printf("Newton:\n  Número de iterações: %d\n  Gradiente final (norma): %.6e\n  Ponto final: [%.6f, %.6f]\n\n",
        max_iter_n, gnorm_n, x[1], x[2])

Gradiente:
  Número de iterações: 1184
  Gradiente final: 9.714477e-07
  Ponto final: [0.999999, 0.999998]

Newton:
  Número de iterações: 5000
  Gradiente final (norma): 5.656854e+00
  Ponto final: [-1.000000, 1.200000]



Para $x_0 = (-1.0, 1.2)$, mais distante, Newton falha em convergir (5000 iterações) porque $H(x)$ longe do mínimo não é positiva definida, enquanto o gradiente ainda converge (1184 iterações).


Observe que, para a função de Rosenbrock

$$
  f(x_1, x_2) = (x_1 - 1)^2 + 10 (x_2 - x_1^2)^2,
$$

a Hessiana no ponto $x_0 = (-1, 1.2)$ é

$$
  H(x) =
  \begin{bmatrix}
  74 & 40 \\
  40 & 20
  \end{bmatrix}.
$$

e o determinante é

$$
  \det(H) = -120 < 0
$$

ou seja, a Hessiana **não é positiva definida** nesse ponto. Isso explica por que o método de Newton não converge a partir desse ponto inicial.


Podemos ajustar o algoritmo de Newton de modo que, caso a Hessiana não seja positiva definida, o método utilize o gradiente como alternativa.

In [38]:
gradient_fallback(g, H) = -g / norm(g)

x, max_iter_n, gnorm_n = newton_backtracking(rosenb, x1, max_iter=5000, fallback=gradient_fallback)
@printf("Newton:\n  Número de iterações: %d\n  Gradiente final (norma): %.6e\n  Ponto final: [%.6f, %.6f]\n\n",
        max_iter_n, gnorm_n, x[1], x[2])

Newton:
  Número de iterações: 12
  Gradiente final (norma): 8.406475e-08
  Ponto final: [1.000000, 1.000000]



## Questão 3

In [31]:
function exact_line_search_quadratic(A, b, x, d)
  numerator = dot(b, d) - dot(d, A * x)
  denominator = dot(d, A * d)

  if denominator <= 0 || abs(denominator) < 1e-14
    return 1e-6   # passo mínimo seguro
  end
  return numerator / denominator
end

exact_line_search_quadratic (generic function with 1 method)

In [32]:
function quadratic(x, A, b)
  f = 0.5 * dot(x, A * x) - dot(b, x)
  g = A * x - b
  H = A
  return f, g, H
end

quadratic (generic function with 1 method)

In [33]:
function householder_matrix(n)
  u = randn(n)
  u /= norm(u)
  U = Matrix(I, n, n) - 2.0 * (u * u')
  return U
end

householder_matrix (generic function with 1 method)

**Gradiente com busca linear exata**

In [53]:
function steepest_descent_exact(A, b, x0; max_iter=2000, tol=1e-6)
  x = copy(x0)
  fvals = Float64[]

  for k = 0:max_iter-1
    f, g, _ = quadratic(x, A, b)
    push!(fvals, f)

    gnorm = norm(g)
    if gnorm <= tol
      return x, k, fvals # Return fvals array here
    end

    d = -g

    # Busca linear exata
    alpha = exact_line_search_quadratic(A, b, x, d)

    x = x + alpha * d
  end

  return x, max_iter, fvals
end

steepest_descent_exact (generic function with 1 method)

**Newton com busca linear exata**

In [54]:
function newton_exact(A, b, x0; max_iter=2000, tol=1e-6)
  x = copy(x0)
  fvals = Float64[]

  for k = 0:max_iter-1
    f, g, H = quadratic(x, A, b)
    push!(fvals, f)

    gnorm = norm(g)
    if gnorm <= tol
      return x, k, fvals
    end

    p = - H \ g

    # passo exato
    alpha = exact_line_search_quadratic(A, b, x, p)

    x = x + alpha * p
  end

  return x, max_iter, fvals
end

newton_exact (generic function with 1 method)

Cria uma instância quadrática $f(x) = \tfrac{1}{2} x^T A x - b^T x$ de dimensão $n$ com uma distribuição específica de autovalores:  
- `:equal`: todos os autovalores iguais (bem condicionada).  
- `:two`: apenas dois valores diferentes (condicionamento moderado).  
- `:spread`: autovalores variados de 1 até 1000 (mal condicionada).

In [43]:
function generate_quadratic_instance(n, case)
  U = householder_matrix(n)

  eigenvalues = zeros(n)

  if case == :equal
    eigenvalues .= 1.0
  elseif case == :two
    eigenvalues[1:div(n,2)] .= 2.0
    eigenvalues[div(n,2)+1:end] .= 5.0
    shuffle!(eigenvalues)
  elseif case == :spread
    eigenvalues .= collect(LinRange(1.0, 1000.0, n))
    shuffle!(eigenvalues)
  end

  Lambda = Diagonal(eigenvalues)
  A = U * Lambda * U'
  A = 0.5 * (A + A')   # garante simetria

  b = randn(n)
  x0 = randn(n)

  return A, b, x0, eigenvalues
end

generate_quadratic_instance (generic function with 1 method)

Neste bloco, aplicamos os métodos de otimização (`steepest_descent_exact` e `newton_exact`) em várias instâncias quadráticas geradas:

- `dimensions = [10, 100, 1000]`: diferentes tamanhos da matriz $A$;
- `cases = [:equal, :two, :spread]`: diferentes distribuições de autovalores para testar o condicionamento.


In [60]:
dimensions = [10, 100, 1000]
cases = [:equal, :two, :spread]

for n in dimensions, case in cases
  A, b, x0, eigenvalues = generate_quadratic_instance(n, case)

  # Gradiente
  t_sd = @elapsed x_sd, k_sd, fvals_sd = steepest_descent_exact(A, b, x0, max_iter=10000)

  # Newton
  t_newt = @elapsed x_newt, k_newt, fvals_newt = newton_exact(A, b, x0, max_iter=10000)

  println("Dimensão: $n, Caso: $case")
  println("  Gradiente:")
  println("    Iterações: $k_sd")
  println("    Tempo: $(round(t_sd, digits=6)) s")
  println("    Ponto final: $x_newt")
  println("  Newton:")
  println("    Iterações: $k_newt")
  println("    Tempo: $(round(t_newt, digits=6)) s")
  println("    Ponto final: $x_newt \n")
end

## ADICIONAR AVALIAÇÃO DE FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF

Dimensão: 10, Caso: equal
  Gradiente:
    Iterações: 1
    Tempo: 2.5e-5 s
    Ponto final: [-1.5595439838004437, -0.6222411765685766, -0.6556141844911572, 0.7366321116615436, -0.3095891776578683, -0.11822677551433172, 0.397221276808584, 1.148495822658993, -0.7620493236575507, 0.8705426485651645]
  Newton:
    Iterações: 1
    Tempo: 2.5e-5 s
    Ponto final: [-1.5595439838004437, -0.6222411765685766, -0.6556141844911572, 0.7366321116615436, -0.3095891776578683, -0.11822677551433172, 0.397221276808584, 1.148495822658993, -0.7620493236575507, 0.8705426485651645] 

Dimensão: 10, Caso: two
  Gradiente:
    Iterações: 11
    Tempo: 3.3e-5 s
    Ponto final: [-0.5824444586641478, 0.009210147374131061, 1.0366796912368774, -0.19164162582341793, 0.13207249390316944, 0.2515976449612658, -0.47804609928019814, 0.4869298634117516, 0.2047968508294442, 0.33352541066389463]
  Newton:
    Iterações: 1
    Tempo: 2.2e-5 s
    Ponto final: [-0.5824444586641478, 0.009210147374131061, 1.0366796912368774,

Observa-se que o método de Newton converge em apenas uma iteração para todos os casos testados, independentemente da dimensão da matriz ou da distribuição dos autovalores. Isso ocorre porque, para uma função quadrática, a direção de Newton aponta exatamente para o mínimo, e o passo exato leva diretamente para ele. O tempo de execução aumenta levemente com a dimensão devido à necessidade de resolver o sistema linear $H p = g$.

Por outro lado, o desempenho do método do gradiente dependente do condicionamento da matriz $A$. Nos casos de autovalores iguais ou com apenas dois distintos, o número de iterações necessárias é relativamente baixo. Mas quando os autovalores estão muito diferentes, o método precisa de mais iterações para convergir. O tempo de processamento cresce à medida que a dimensão da matriz aumenta.